In [31]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#Clean up code
#add iterative testing
#create test result charts
#output results and charts to folders
#add diagonal to MAE
#create MFE
#find perfect trades to determine optimal moving average

In [1]:
%matplotlib qt
#%matplotlib inline

#import statements 
import xlsxwriter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.dates as dates
import datetime
import time
import csv
import sys

In [70]:

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

class Test:
    def __init__(self, acct_bal, equity):
        self.init_bal = acct_bal #initial balance
        self.acct_bal = acct_bal #ongoing account balance
        self.equity = equity #unrealized p/L
        self.buy_status = 1 #these aren't really doing anything
        self.sell_status = 0
        self.trades = 1
        self.profit = 0
        self.units = 0
        self.trade_profit = 0
        self.activity = np.zeros((1,), dtype=[('Trade', 'int32'), ('Ask', 'float64'), ('Ask_TID', 'int32'), 
                                              ('Units','int32'), ('Bid', 'float64'), ('Bid_TID','int32'), 
                                              ('Profit', 'float64'), ('Stop_Loss_Floor', 'float64'), ('Stop_Loss_Bid', 'float64'), 
                                              ('Stop_Loss_Bid_TID','int32'), ('Max_DD', 'float64'), ('Max_DD_TID', 'int32'), 
                                              ('Max_DD_Bid', 'float64'), ('Stop_Loss_Max_DD', 'float64'), 
                                              ('Stop_Loss_Max_DD_TID', 'int32'),('Stop_Loss_Max_DD_Bid', 'float64')])
        self.new_activity = np.empty_like(self.activity)
        self.buy_start = 0
        self.buy_end = 0
        self.stop_loss = None #define stop loss for reporting
        
        
    def buy(self, amt, i):
        #you buy at the ask and sell at the bid (quoted from perspective of market maker)
        
        #pull arr_analysis tuple to assign variable values
        ask = arr_analysis[i]['Ask']
        TID = arr_analysis[i]['TID']
        self.buy_start = i
        
        
        #calculate the buy
        self.units = int((amt)/ask) #calcluate units per trade at amount
        self.equity += (self.units * ask) #set equity to unrealized p/l (this is more accurately "position")
        self.acct_bal -= self.equity #subtract equity from ongoing account balance
        
        #update activity
        self.new_activity[0]['Trade']= self.trades
        self.new_activity[0]['Ask']= ask
        self.new_activity[0]['Ask_TID']= TID
        self.new_activity[0]['Units']= self.units
                
        #set flags
        self.buy_status = 0
        self.sell_status = 1

        
    def sell(self, i, stop_loss = None):
        
        self.stop_loss = stop_loss
        
        #pull arr_analysis tuple to assign variable values
        bid = arr_analysis[i]['Bid'] #pull the current Bid (what you can sell for)
        TID = arr_analysis[i]['TID'] #pull the TID for the current Bid
        self.buy_end = i + 1 #Define the end of the array as the current row plus 1 
        
        #determine the maximum drawdown
        ask = self.new_activity['Ask'] #the price you bought at
        ask_tid = self.new_activity['Ask_TID'] #the ID of the tick for the price you bought at
                
        arr_sub_analysis = arr_analysis[self.buy_start:self.buy_end]
        
        min_bid = np.amin(arr_sub_analysis['Bid']) #the lowest purchase price in the range of your purchase to sale
        
        arr_min_bid_tuple = arr_sub_analysis[np.argmin(arr_sub_analysis['Bid'])] #find the index for the lowest bid
        min_bid_tid = arr_min_bid_tuple['TID'] #get the TID for the lowest bid
        
        self.new_activity[0]['Max_DD'] = (min_bid * self.units) - self.equity #calculate the max drawdown amount
        self.new_activity[0]['Max_DD_TID'] = min_bid_tid #pull the TID for the max dd
        self.new_activity[0]['Max_DD_Bid'] = min_bid #pull the bid associated with the max dd
        self.new_activity[0]['Stop_Loss_Max_DD'] = (min_bid * self.units) - self.equity #default stop loss max dd amount for graphing
        self.new_activity[0]['Stop_Loss_Max_DD_TID'] = min_bid_tid #default stop loss max dd tid for graphing
        self.new_activity[0]['Stop_Loss_Max_DD_Bid'] = min_bid #default stop loss max dd bid for graphing

        
        if self.stop_loss != None: #only calculate stop loss values if stop loss is specified        
            stop_loss_floor = (self.equity-self.stop_loss)/self.units #calculate the Bid value of your loss stop
            self.new_activity[0]['Stop_Loss_Floor'] = stop_loss_floor

            #get array of all ticks where bid is gte to stop loss floor
            arr_gte_stop_loss = np.squeeze(np.take(arr_sub_analysis, np.where(arr_sub_analysis['Bid']>= stop_loss_floor)))
                
        if stop_loss == None or stop_loss > abs(self.new_activity[0]['Max_DD']):
            self.new_activity[0]['Bid'] = bid #assign the parameter bid
            self.new_activity[0]['Bid_TID'] = TID #assign the parameter bid TID
            self.new_activity[0]['Stop_Loss_Floor'] = 0
            self.new_activity[0]['Stop_Loss_Bid'] = 0
            self.new_activity[0]['Stop_Loss_Bid_TID'] = 0
            
            self.calc_sale()
                                
        else:
            try:
                if arr_gte_stop_loss.size < 2: #handle data sets where bid is never >= min bid. We actuially want to skip stop loss and just use standard logic for realism
                    self.new_activity[0]['Stop_Loss_Bid'] = 0
                    self.new_activity[0]['Stop_Loss_Bid_TID'] = 0
                    self.new_activity[0]['Bid'] = bid #assign the parameter bid
                    self.new_activity[0]['Bid_TID'] = TID #assign the parameter bid TID
                    self.calc_sale()
                
                else:
                    stop_loss_bid_index = np.argmin(arr_gte_stop_loss['TID'])#get the index of lowest acceptable bid
                    stop_loss_TID = np.amin(arr_gte_stop_loss['TID'])
                    arr_stop_loss_bid = arr_gte_stop_loss[stop_loss_bid_index] #the row of the lowest purchase price in the range of your purchase to sale
                    stop_loss_bid = arr_stop_loss_bid['Bid'] #the ID of the lowest purchase price
                
                    self.new_activity[0]['Stop_Loss_Bid'] = stop_loss_bid
                    self.new_activity[0]['Stop_Loss_Bid_TID'] = stop_loss_TID
                    self.new_activity[0]['Bid'] = stop_loss_bid
                    self.new_activity[0]['Bid_TID'] = stop_loss_TID
                    
                    self.new_activity[0]['Stop_Loss_Max_DD'] = (stop_loss_bid * self.units) - self.equity #calculate the max drawdown amount
                    self.new_activity[0]['Stop_Loss_Max_DD_TID'] = stop_loss_TID #this is actually the lowest bid and not the dd bid
                    self.new_activity[0]['Stop_Loss_Max_DD_Bid'] = stop_loss_bid #this is actually the lowest bid and not the dd bid


                    self.calc_sale()
                            
            except:
                print("Unexpected error:", sys.exc_info())
                #print(err)                
                print("The current trade is:", self.trades)
                print("The current ask is:", self.new_activity[0]['Ask'])
                print("The current equity is:", self.equity)
                print("The current units are:", self.units)
                print("The current min bid is:", min_bid)
                print("The min bid criteria is:", max_dd_criteria)
                print("Array analysis starts at:", self.buy_start, "and ends at:", self.buy_end)
                sys.exit('Exiting because of error')
        
        #calc_sale()
        
        
    def calc_sale(self):
        #calculate the sale
        self.acct_bal = self.acct_bal + (self.units * self.new_activity[0]['Bid']) #add realized p/l to account balance        
        self.trade_amt = self.units * self.new_activity[0]['Bid'] #calculate trade value
        self.trade_profit = (self.units * self.new_activity[0]['Bid']) - self.equity #calculate the trade profit
        self.profit += self.trade_profit #update running profit
        self.equity = 0 #need to change this so it takes bid into accountif stop_loss == None or stop_loss < abs(self.new_activity[0]['Max_DD']):
        
        #update activity
        #self.new_activity[0]['Bid'] = bid
        #self.new_activity[0]['Bid_TID'] = TID
        self.new_activity[0]['Profit']=self.trade_profit
        self.activity = np.append(self.activity, self.new_activity)
        
        #set flags
        self.buy_status = 1
        self.sell_status = 0
        self.trades += 1
    
    def acct_status(self):
        print("The current account balance is:{0:.3f}".format(self.acct_bal), 
              "The current equity is:{0:.3f}".format(self.equity),
              "The profit for this trade was:{0:.3f}".format(self.trade_profit),
              "The current profit is:{0:.3f}".format(self.profit), 
              #"The drawdown for this trade is:{0:.3f}".format(self.activity[0]['Max_DD']),
              self.activity,
              sep='\n')

In [2]:
%%time

#Data wrangling

#Data import
df_tick_data = pd.read_csv('C:\\Users\\pebaqu\\Downloads\\USDJPY-2018-01.csv', parse_dates=True )
df_tick_data['TTime']=df_tick_data['TTime'].apply(
    lambda x: np.datetime64(datetime.datetime.strptime(x, '%Y%m%d %H:%M:%S.%f'),unit = 'ms'))

#Data sorting and base column creation
df_tick_data_sorted=df_tick_data.sort_values(by=['TTime'])
df_tick_data_sorted['TID'] = range(0, len(df_tick_data_sorted))
df_tick_data_sorted.set_index('TID')
df_tick_data_sorted['Bid_Ask_Spread']=df_tick_data_sorted['Bid'] - df_tick_data_sorted['Ask']
df_tick_data_sorted.index = df_tick_data_sorted['TID']

#Base dataset creation
df_analysis=pd.DataFrame(df_tick_data_sorted)

#Stat column creation
df_analysis['Bid_Fast_Avg']=df_analysis.Bid.rolling(200).mean()
df_analysis['Prev_Bid_Fast_Avg']=df_analysis['Bid_Fast_Avg'].shift()
df_analysis['Bid_Slow_Avg']=df_analysis.Bid.rolling(20).mean()
df_analysis['Prev_Bid_Slow_Avg']=df_analysis['Bid_Slow_Avg'].shift()
df_analysis['Ask_Fast_Avg']=df_analysis.Ask.rolling(200).mean()
df_analysis['Prev_Ask_Fast_Avg']=df_analysis['Ask_Fast_Avg'].shift()
df_analysis['Ask_Slow_Avg']=df_analysis.Ask.rolling(20).mean()
df_analysis['Prev_Ask_Slow_Avg']=df_analysis['Ask_Slow_Avg'].shift()
df_analysis['Spread_Fast_Avg']=df_analysis.Bid_Ask_Spread.rolling(200).mean()
df_analysis['Prev_Spread_Fast_Avg']=df_analysis['Spread_Fast_Avg'].shift()
df_analysis['Spread_Slow_Avg']=df_analysis.Bid_Ask_Spread.rolling(20).mean()
df_analysis['Prev_Spread_Slow_Avg']=df_analysis['Spread_Slow_Avg'].shift()

#Final analysis dataset creation
df_analysis_final=df_analysis.dropna()

#analysis array creation
arr_ip = [tuple(i) for i in df_analysis_final.values]

dtyp = np.dtype(list(zip(df_analysis_final.dtypes.index, df_analysis_final.dtypes)))

arr_analysis = np.array(arr_ip, dtype=dtyp)



Wall time: 1min 36s


In [87]:
%%time

#Test on actual data using array. Much faster, 12.1 seconds (without stop loss)
#TTime isn't unique! 2018-01-01 22:13:03.523000 USDJPY

test2 = Test(50000,0)
test2.acct_status()

i=0

for x in np.ndenumerate(arr_analysis):
    
    if (arr_analysis[i]['Bid_Fast_Avg'] < arr_analysis[i]['Ask_Slow_Avg'] and test2.buy_status == 1):
        test2.buy(1000, i) #just pass it i

    elif (arr_analysis[i]['Bid_Fast_Avg'] > arr_analysis[i]['Ask_Slow_Avg'] and test2.sell_status == 1):    
        test2.sell(i) #just pass it i and stop loss
    
    i += 1
    
test2.acct_status()

The current account balance is:50000.000
The current equity is:0.000
The profit for this trade was:0.000
The current profit is:0.000
[(0, 0.000, 0, 0, 0.000, 0, 0.000, 0.000, 0.000, 0, 0.000, 0, 0.000, 0.000, 0, 0.000)]
The current account balance is:49697.297
The current equity is:0.000
The profit for this trade was:0.126
The current profit is:-302.703
[(    0, 0.000,       0, 0, 0.000,       0, 0.000, 0.000, 0.000, 0, 0.000,       0, 0.000, 0.000,       0, 0.000)
 (    1, 112.661,     200, 8, 112.603,    5325, -0.464, 0.000, 0.000, 0, -0.728,    1456, 112.570, -0.728,    1456, 112.570)
 (    2, 112.610,    5344, 8, 112.653,    8022, 0.344, 0.000, 0.000, 0, -0.088,    5395, 112.599, -0.088,    5395, 112.599)
 ...
 (12172, 109.156, 3184423, 9, 109.202, 3184755, 0.414, 0.000, 0.000, 0, -0.063, 3184423, 109.149, -0.063, 3184423, 109.149)
 (12173, 109.184, 3185005, 9, 109.171, 3185053, -0.117, 0.000, 0.000, 0, -0.117, 3185053, 109.171, -0.117, 3185053, 109.171)
 (12174, 109.183, 3185065, 

In [88]:
%%time

#convert activity array to dataframe for graphing

df_activity = pd.DataFrame(data=test2.activity[1:,],    # values
            columns=test2.activity.dtype.names)  # 1st row as the column names

df_activity['Bars'] = df_activity['Bid_TID'] - df_activity['Ask_TID']

display(df_activity)

,Trade,Ask,Ask_TID,Units,Bid,Bid_TID,Profit,Stop_Loss_Floor,Stop_Loss_Bid,Stop_Loss_Bid_TID,Max_DD,Max_DD_TID,Max_DD_Bid,Stop_Loss_Max_DD,Stop_Loss_Max_DD_TID,Stop_Loss_Max_DD_Bid,Bars
0,1,112.661,200,8,112.603,5325,-0.464,0.0,0.0,0,-0.728,1456,112.570,-0.728,1456,112.570,5125
1,2,112.610,5344,8,112.653,8022,0.344,0.0,0.0,0,-0.088,5395,112.599,-0.088,5395,112.599,2678
2,3,112.672,8042,8,112.650,8136,-0.176,0.0,0.0,0,-0.224,8132,112.644,-0.224,8132,112.644,94
3,4,112.649,8218,8,112.735,11003,0.688,0.0,0.0,0,-0.176,8504,112.627,-0.176,8504,112.627,2785
4,5,112.740,11023,8,112.735,11043,-0.040,0.0,0.0,0,-0.048,11041,112.734,-0.048,11041,112.734,20
5,6,112.740,11046,8,112.729,11097,-0.088,0.0,0.0,0,-0.088,11095,112.729,-0.088,11095,112.729,51
6,7,112.733,11185,8,112.726,11196,-0.056,0.0,0.0,0,-0.056,11185,112.726,-0.056,11185,112.726,11
7,8,112.733,11204,8,112.740,12826,0.056,0.0,0.0,0,-0.096,12291,112.721,-0.096,12291,112.721,1622
8,9,112.752,12841,8,112.747,13131,-0.040,0.0,0.0,0,-0.072,13115,112.743,-0.072,13115,112.743,290
9,10,112.751,13137,8,112.747,13145,-0.032,0.0,0.0,0,-0.048,13138,112.745,-0.048,13138,112.745,8


Wall time: 66.7 ms


In [76]:
%%time
#Maximum Adverse Event - for debugging

positive1 = df_activity[df_activity['Profit']>=0]
positive = positive1[['Trade','Max_DD','Profit']].values
negative1 = df_activity[df_activity['Profit']<-.08]
negative = negative1[['Trade','Max_DD','Profit']].values

f, ax = plt.subplots(1)
'''
for i in range(len(positive)):
    pointRefNumber = int(positive[i][0])
    xPoint =  abs(positive[i][1])
    yPoint =  abs(positive[i][2])
    ax.plot(xPoint, yPoint, marker="+", c="g")
    ax.annotate(pointRefNumber, (xPoint, yPoint), fontsize=12)
'''
for i in range(len(negative)):
    pointRefNumber = int(negative[i][0])
    xPoint =  abs(negative[i][1])
    yPoint =  abs(negative[i][2])
    ax.plot(xPoint, yPoint, marker="o", c="r")
    ax.annotate(pointRefNumber, (xPoint, yPoint), fontsize=12)
ax.plot(min(ax.get_ylim(),ax.get_xlim()) , min(ax.get_ylim(),ax.get_xlim()), ls="--", c=".3")
plt.axvline(x=test2.stop_loss)
ax.grid(color='g', linestyle='dashed', linewidth=1)
ax.set_xlim(xmin=0)
ax.set_ylim(ymin=0)
ax.set_title("Maximum Adverse Event \n Stop Loss:" +str(test2.stop_loss))


TypeError: '>' not supported between instances of 'float' and 'NoneType'

In [82]:
%%time

#Maximum Adverse Event

positive = df_activity[df_activity['Profit']>=0]
negative = df_activity[df_activity['Profit']<0]

f, ax = plt.subplots(figsize=(20,10))
ax.scatter(abs(positive['Stop_Loss_Max_DD']), abs(positive['Profit']), marker="+", c="g")
ax.scatter(abs(negative['Stop_Loss_Max_DD']), abs(negative['Profit']), marker="o", c="r")

ax.plot(min(ax.get_ylim(),ax.get_xlim()) , min(ax.get_ylim(),ax.get_xlim()), ls="--", c=".3")
ax.grid(color='g', linestyle='dashed', linewidth=1)
ax.set_xlim(xmin=0)
ax.set_ylim(ymin=0)
plt.axvline(x=test2.stop_loss)
plt.xlabel('Max Drawdown')
plt.ylabel('Profit ($)')
ax.set_title("Maximum Adverse Event \n Stop Loss:" +str(test2.stop_loss))
plt.savefig('C:\\Users\\pebaqu\\Desktop\Personal\\Python\\JupyterExports\\TestGraph.png')

Wall time: 367 ms


In [94]:
%%time
#Underwater Equity Curve

plt.plot(df_activity['Trade'],df_activity['Profit'], color='none', linewidth=0)
plt.fill_between(df_activity['Trade'], df_activity['Profit'], where=df_activity['Profit']>=0, facecolor='green', interpolate=True)
plt.fill_between(df_activity['Trade'], df_activity['Profit'], where=df_activity['Profit']<0, facecolor='red', interpolate=True)
plt.axhline(y=-.1)
plt.title("Underwater Equity Curve \n Stop Loss:" +str(test2.stop_loss))
plt.show

Wall time: 444 ms


In [86]:
display(df_activity.sort_values(by=['Profit']))

,Trade,Ask,Ask_TID,Units,Bid,Bid_TID,Profit,Stop_Loss_Floor,Stop_Loss_Bid,Stop_Loss_Bid_TID,Max_DD,Max_DD_TID,Max_DD_Bid,Stop_Loss_Max_DD,Stop_Loss_Max_DD_TID,Stop_Loss_Max_DD_Bid,Bars
1568,1569,113.134,452833,8,113.021,452855,-0.904,113.121500,0.0,0,-0.976,452833,113.012,-0.976,452833,113.012,22
1382,1383,113.128,402846,8,113.016,402875,-0.896,113.115500,0.0,0,-0.960,402861,113.008,-0.960,402861,113.008,29
4932,4933,110.457,1359807,9,110.375,1359967,-0.738,110.445889,0.0,0,-0.738,1359967,110.375,-0.738,1359967,110.375,160
356,357,112.332,116486,8,112.242,116537,-0.720,112.319500,0.0,0,-0.784,116531,112.234,-0.784,116531,112.234,51
9989,9990,108.767,2646102,9,108.703,2646107,-0.576,108.755889,0.0,0,-0.594,2646106,108.701,-0.594,2646106,108.701,5
12010,12011,109.258,3137962,9,109.199,3137975,-0.531,109.246889,0.0,0,-0.711,3137967,109.179,-0.711,3137967,109.179,13
12008,12009,109.266,3137785,9,109.207,3137841,-0.531,109.254889,0.0,0,-0.531,3137841,109.207,-0.531,3137841,109.207,56
12156,12157,109.266,3177292,9,109.207,3180946,-0.531,109.254889,0.0,0,-0.594,3180933,109.200,-0.594,3180933,109.200,3654
361,362,112.281,121285,8,112.215,121330,-0.528,112.268500,0.0,0,-0.528,121330,112.215,-0.528,121330,112.215,45
12009,12010,109.259,3137883,9,109.204,3137910,-0.495,109.247889,0.0,0,-0.531,3137901,109.200,-0.531,3137901,109.200,27


In [51]:
#understanding stop loss behavior

df_stop_loss = df_activity[df_activity['Max_DD']<= -.1]

display(df_stop_loss)

,Trade,Ask,Ask_TID,Units,Bid,Bid_TID,Profit,Stop_Loss_Floor,Stop_Loss_Bid,Stop_Loss_Bid_TID,Max_DD,Max_DD_TID,Max_DD_Bid,Bars
0,1,112.661,200,8,112.603,5325,-0.096,112.648500,112.649,398,-0.728,1456,112.570,5125
2,3,112.672,8042,8,112.650,8136,-0.072,112.659500,112.663,8042,-0.224,8132,112.644,94
3,4,112.649,8218,8,112.735,11003,-0.096,112.636500,112.637,8222,-0.176,8504,112.627,2785
17,18,112.785,15021,8,112.772,15360,-0.040,112.772500,112.780,15021,-0.104,15350,112.772,339
18,19,112.777,15393,8,112.760,15922,-0.040,112.764500,112.772,15393,-0.152,15921,112.758,529
19,20,112.780,15948,8,112.759,16297,-0.040,112.767500,112.775,15948,-0.176,16292,112.758,349
20,21,112.764,16394,8,112.754,16547,-0.056,112.751500,112.757,16394,-0.104,16540,112.751,153
23,24,112.753,17540,8,112.738,17601,-0.032,112.740500,112.749,17540,-0.120,17601,112.738,61
26,27,112.739,19575,8,112.729,19620,-0.032,112.726500,112.735,19575,-0.104,19616,112.726,45
28,29,112.737,19685,8,112.715,19743,-0.064,112.724500,112.729,19685,-0.176,19743,112.715,58


In [48]:
#understanding stop loss behavior

df_stop_loss = df_activity[df_activity['Max_DD']<= -.1]

display(df_stop_loss)

,Trade,Ask,Ask_TID,Units,Bid,Bid_TID,Profit,Stop_Loss_Floor,Stop_Loss_Bid,Stop_Loss_Bid_TID,Max_DD,Max_DD_TID,Max_DD_Bid,Bars
0,1,112.661,200,8,112.603,5325,-0.464,0.0,0.0,0,-0.728,1456,112.570,5125
2,3,112.672,8042,8,112.650,8136,-0.176,0.0,0.0,0,-0.224,8132,112.644,94
3,4,112.649,8218,8,112.735,11003,0.688,0.0,0.0,0,-0.176,8504,112.627,2785
17,18,112.785,15021,8,112.772,15360,-0.104,0.0,0.0,0,-0.104,15350,112.772,339
18,19,112.777,15393,8,112.760,15922,-0.136,0.0,0.0,0,-0.152,15921,112.758,529
19,20,112.780,15948,8,112.759,16297,-0.168,0.0,0.0,0,-0.176,16292,112.758,349
20,21,112.764,16394,8,112.754,16547,-0.080,0.0,0.0,0,-0.104,16540,112.751,153
23,24,112.753,17540,8,112.738,17601,-0.120,0.0,0.0,0,-0.120,17601,112.738,61
26,27,112.739,19575,8,112.729,19620,-0.080,0.0,0.0,0,-0.104,19616,112.726,45
28,29,112.737,19685,8,112.715,19743,-0.176,0.0,0.0,0,-0.176,19743,112.715,58


In [5]:
#troubleshooting "too many indices error" test trade 358 (should not produce error)

start = 116563 - 200
stop = 118729 - 200

ask = 112.304
units = 8
stop_loss = .1
equity = units * ask

print("equity is:", equity)


arr_test = arr_analysis[start:stop]


max_dd_criteria = (equity-stop_loss)/units #calculate the Bid value of your loss stop
print("max_dd_criteria is:" , max_dd_criteria)


max_dd_min_bid = np.empty_like(arr_analysis) #create the empty array for your analysis

max_dd_min_bid = arr_analysis[start:stop] #fill the array with your start and stop
print("max_dd_min_bid dimensions are:" , max_dd_min_bid.ndim)

#need to apply a test to see what the shape is after the squeeze
arr_max_dd_min_bid = np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria)))
print("arr_max_dd_min_bid is:" , arr_max_dd_min_bid)
print("arr_max_dd_min_bid shape is:", arr_max_dd_min_bid.shape)
print("arr_max_dd_min_bid size:", arr_max_dd_min_bid.size)


#handle data sets where bid is never >= min bid
if arr_max_dd_min_bid.size == 0:
    arr_max_dd_min_bid = max_dd_min_bid[np.argmax(max_dd_min_bid['Bid'])]
    min_bid = arr_max_dd_min_bid['Bid']
    min_bid_tid = arr_max_dd_min_bid['TID']

else:


    min_bid = np.amin(arr_max_dd_min_bid['Bid']) #the lowest purchase price in the range of your purchase to sale
    print("min bid is:", min_bid)

    min_index = int(np.argmin(arr_max_dd_min_bid['Bid']))
    print("min_index is:", min_index)
    min_bid_tuple = arr_max_dd_min_bid[min_index] #the row of the lowest purchase price in the range of your purchase to sale
    min_bid_tid = min_bid_tuple['TID'] #the ID of the lowest purchase price


equity is: 898.432
max_dd_criteria is: 112.2915
max_dd_min_bid dimensions are: 1
arr_max_dd_min_bid is: [('USD/JPY', '2018-01-02T22:24:31.428000000', 112.304, 112.33 , 116585, -0.026, 112.29999 , 112.300085, 112.28725, 112.2859 , 112.318115, 112.31812 , 112.31025, 112.30895, -0.018125, -0.018035, -0.023  , -0.02305)
 ('USD/JPY', '2018-01-02T22:24:31.481000000', 112.304, 112.334, 116586, -0.03 , 112.299915, 112.29999 , 112.28855, 112.28725, 112.31813 , 112.318115, 112.31175, 112.31025, -0.018215, -0.018125, -0.0232 , -0.023  )
 ('USD/JPY', '2018-01-02T22:24:31.752000000', 112.315, 112.334, 116587, -0.019, 112.299875, 112.299915, 112.29035, 112.28855, 112.318145, 112.31813 , 112.31325, 112.31175, -0.01827 , -0.018215, -0.0229 , -0.0232 )
 ('USD/JPY', '2018-01-02T22:24:31.754000000', 112.304, 112.334, 116588, -0.03 , 112.2998  , 112.299875, 112.2914 , 112.29035, 112.31816 , 112.318145, 112.31475, 112.31325, -0.01836 , -0.01827 , -0.02335, -0.0229 )
 ('USD/JPY', '2018-01-02T22:24:32.130000

In [ ]:
df_activity[355:360]

In [21]:
#troubleshooting "too many indices error" trade 357
trade = 1383
start = 402846 - 200
stop = 402876 - 200

ask = 113.128
units = 8
stop_loss = .1
equity = units * ask

print("equity is:", equity)


arr_test = arr_analysis[start:stop]

#max_dd_criteria
stop_loss_bid = (equity-stop_loss)/units #calculate the Bid value of your loss stop
print("stop loss bid is:" , stop_loss_bid)


arr_sub_analysis = np.empty_like(arr_analysis) #create the empty array for your analysis

arr_sub_analysis = arr_analysis[start:stop] #fill the array with your start and stop
print("arr_sub_analysis dimensions are:" , arr_sub_analysis.ndim)

#need to apply a test to see what the shape is after the squeeze
arr_gte_stop_loss = np.squeeze(np.take(arr_sub_analysis, np.where(arr_sub_analysis['Bid']>= stop_loss_bid)))
print("arr_gte_stop_loss is:" , arr_gte_stop_loss)
print("arr_gte_stop_loss shape is:", arr_gte_stop_loss.shape)
print("arr_gte_stop_loss size:", arr_gte_stop_loss.size)


#handle data sets where bid is never >= min bid. We actuially want to skip stop loss and just use standard logic for realism
if arr_gte_stop_loss.size < 2:
    arr_stop_loss = arr_sub_analysis[np.argmin(arr_sub_analysis['Bid'])] #take the lowest bid for worst case scenario
    stop_loss_bid = arr_stop_loss['Bid']
    print("single tuple stop_loss is:", stop_loss_bid)
    min_bid_tid = arr_stop_loss['TID']

else:

    stop_loss = np.amin(arr_gte_stop_loss['Bid']) #the lowest purchase price in the range of your purchase to sale
    print("multiple tuple stop loss is:", stop_loss)

    stop_loss_bid_index = np.argmin(arr_gte_stop_loss['Bid'])
    print("multiple tuple stop loss index is:", stop_loss_bid_index)
    arr_stop_loss_bid = arr_gte_stop_loss[stop_loss_bid_index] #the row of the lowest purchase price in the range of your purchase to sale
    stop_loss_bid_tid = arr_stop_loss_bid['TID'] #the ID of the lowest purchase price


##DATA EXPORT
df_export = pd.DataFrame(data=arr_test,    # values
columns=arr_analysis.dtype.names)

#Export sample data
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\Trade '+str(trade)+' analysis.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()
##END DATA EXPORT


equity is: 905.024
stop loss bid is: 113.1155
arr_sub_analysis dimensions are: 1
arr_gte_stop_loss is: []
arr_gte_stop_loss shape is: (0,)
arr_gte_stop_loss size: 0
single tuple stop_loss is: 113.008


In [20]:
start = 115723 - 200 #have to set the array index to 200 less because of the drop na
stop = 120934 - 200

df_export = pd.DataFrame(data=arr_analysis[0:100],    # values
            columns=arr_analysis.dtype.names)
        
#Export sample data
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\Test.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()
##END DATA EXPORTarr_analysis[115723:120936]

In [13]:
df_argmin = pd.DataFrame(data=arr_analysis[118776:118779,],    # values
            columns=arr_analysis.dtype.names)

display(df_argmin)

arr_sub_argmin = arr_analysis[118772:118775,]

print(arr_sub_argmin)

#test = np.empty_like(arr_analysis)

np.argmin(arr_sub_argmin['Bid'])
#print(test)

,Pair,TTime,Bid,Ask,TID,Bid_Ask_Spread,Bid_Fast_Avg,Prev_Bid_Fast_Avg,Bid_Slow_Avg,Prev_Bid_Slow_Avg,Ask_Fast_Avg,Prev_Ask_Fast_Avg,Ask_Slow_Avg,Prev_Ask_Slow_Avg,Spread_Fast_Avg,Prev_Spread_Fast_Avg,Spread_Slow_Avg,Prev_Spread_Slow_Avg
0,USD/JPY,2018-01-02 22:36:38.505,112.290,112.304,118976,-0.014,112.26965,112.269535,112.2880,112.28725,112.292255,112.292160,112.30170,112.3017,-0.022605,-0.022625,-0.01370,-0.01445
1,USD/JPY,2018-01-02 22:36:38.557,112.289,112.304,118977,-0.015,112.26976,112.269650,112.2885,112.28800,112.292345,112.292255,112.30190,112.3017,-0.022585,-0.022605,-0.01340,-0.01370
2,USD/JPY,2018-01-02 22:36:38.823,112.289,112.303,118978,-0.014,112.26987,112.269760,112.2885,112.28850,112.292425,112.292345,112.30205,112.3019,-0.022555,-0.022585,-0.01355,-0.01340


[('USD/JPY', '2018-01-02T22:36:35.910000000', 112.289, 112.303, 118972, -0.014, 112.269, 112.269, 112.286, 112.285, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.016)
 ('USD/JPY', '2018-01-02T22:36:36.758000000', 112.289, 112.304, 118973, -0.015, 112.269, 112.269, 112.286, 112.286, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.015)
 ('USD/JPY', '2018-01-02T22:36:36.809000000', 112.289, 112.303, 118974, -0.014, 112.269, 112.269, 112.287, 112.286, 112.292, 112.292, 112.302, 112.301, -0.023, -0.023, -0.015, -0.015)]


0

In [15]:
df_argmin = pd.DataFrame(data=arr_analysis[118772:118775,],    # values
            columns=arr_analysis.dtype.names)

display(df_argmin)

arr_sub_argmin = arr_analysis[118772:118775,]

print(arr_sub_argmin)

#test = np.empty_like(arr_analysis)

np.argmin(arr_sub_argmin['Bid'])
#print(test)

min_bid_tuple = arr_sub_argmin[np.argmin(arr_sub_argmin['Bid'])]
print(min_bid_tuple)


'''
min_bid_tuple = arr_max_dd_min_bid[np.argmin(arr_max_dd_min_bid['Bid'])]
too many indices for the array
'''







max_dd_criteria = 112.2775 #calculate the Bid value of your loss stop


max_dd_min_bid = np.empty_like(arr_analysis) #create the empty array for your analysis

max_dd_min_bid = arr_analysis[118772:118775] #fill the array with your start and stop
'''
##DATA EXPORT
df_export = pd.DataFrame(data=max_dd_min_bid,    # values
columns=max_dd_min_bid.dtype.names)

#Export sample data
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\max_dd_min_bid' + str(self.trades) + '.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()
##END DATA EXPORT
'''

arr_max_dd_min_bid = np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria)))
print(arr_max_dd_min_bid)

'''

##DATA EXPORT
df_export = pd.DataFrame.from_records(data=arr_max_dd_min_bid)#,    # values
#columns=arr_max_dd_min_bid.dtype.names)

#Export sample data
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\Users\\pebaqu\\Desktop\\Personal\\Python\\JupyterExports\\arr_max_dd_min_bid' + str(self.trades) + '.xlsx', engine='xlsxwriter')

#Convert the dataframe to an XlsxWriter Excel object.
df_export.to_excel(writer, sheet_name='Sheet1')
#Close the Pandas Excel writer and output the Excel file.
writer.save()
##END DATA EXPORT
'''

min_bid = np.amin(arr_max_dd_min_bid['Bid']) #the lowest purchase price in the range of your purchase to sale

min_bid_tuple = arr_max_dd_min_bid[np.argmin(arr_max_dd_min_bid['Bid'])] #the row of the lowest purchase price in the range of your purchase to sale
min_bid_tid = min_bid_tuple['TID'] #the ID of the lowest purchase price




,Pair,TTime,Bid,Ask,TID,Bid_Ask_Spread,Bid_Fast_Avg,Prev_Bid_Fast_Avg,Bid_Slow_Avg,Prev_Bid_Slow_Avg,Ask_Fast_Avg,Prev_Ask_Fast_Avg,Ask_Slow_Avg,Prev_Ask_Slow_Avg,Spread_Fast_Avg,Prev_Spread_Fast_Avg,Spread_Slow_Avg,Prev_Spread_Slow_Avg
0,USD/JPY,2018-01-02 22:36:35.910,112.289,112.303,118972,-0.014,112.269175,112.269055,112.2859,112.28545,112.29191,112.291845,112.30120,112.3011,-0.022735,-0.022790,-0.01530,-0.01565
1,USD/JPY,2018-01-02 22:36:36.758,112.289,112.304,118973,-0.015,112.269295,112.269175,112.2864,112.28590,112.29199,112.291910,112.30140,112.3012,-0.022695,-0.022735,-0.01500,-0.01530
2,USD/JPY,2018-01-02 22:36:36.809,112.289,112.303,118974,-0.014,112.269415,112.269295,112.2868,112.28640,112.29207,112.291990,112.30155,112.3014,-0.022655,-0.022695,-0.01475,-0.01500


[('USD/JPY', '2018-01-02T22:36:35.910000000', 112.289, 112.303, 118972, -0.014, 112.269, 112.269, 112.286, 112.285, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.016)
 ('USD/JPY', '2018-01-02T22:36:36.758000000', 112.289, 112.304, 118973, -0.015, 112.269, 112.269, 112.286, 112.286, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.015)
 ('USD/JPY', '2018-01-02T22:36:36.809000000', 112.289, 112.303, 118974, -0.014, 112.269, 112.269, 112.287, 112.286, 112.292, 112.292, 112.302, 112.301, -0.023, -0.023, -0.015, -0.015)]
('USD/JPY', '2018-01-02T22:36:35.910000000', 112.289, 112.303, 118972, -0.014, 112.269, 112.269, 112.286, 112.285, 112.292, 112.292, 112.301, 112.301, -0.023, -0.023, -0.015, -0.016)


"\nmin_bid_tuple = arr_max_dd_min_bid[np.argmin(arr_max_dd_min_bid['Bid'])]\ntoo many indices for the array\n"

In [16]:
#figure out why trade 357 was bombing out - there was no bid >= to min bid

start = 118772 - 200
stop = 118775 - 200

max_dd_criteria = 112.2775 #calculate the Bid value of your loss stop

arr_max_dd_min_bid = np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria)))

max_dd_min_bid = np.empty_like(arr_analysis) #create the empty array for your analysis

max_dd_min_bid = arr_analysis[start:stop] #fill the array with your start and stop

arr_max_dd_min_bid = np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria)))

print(arr_max_dd_min_bid.shape)
print(arr_max_dd_min_bid.strides)
print(arr_max_dd_min_bid.size)
print(max_dd_min_bid[['Bid','Ask','TID']])
print(arr_max_dd_min_bid.ndim, arr_max_dd_min_bid.size)
print(max_dd_min_bid.ndim, max_dd_min_bid.size)
#print(np.argmin(arr_max_dd_min_bid['Bid']))

#min_bid_tuple = arr_max_dd_min_bid[np.argmin(arr_max_dd_min_bid['Bid'])]

'''
if np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid['Bid']>= max_dd_criteria))).size == 0:
    print(np.argmax(max_dd_min_bid['Bid']))
    arr_max_dd_min_bid = max_dd_min_bid[np.argmax(max_dd_min_bid['Bid'])]
    min_bid = arr_max_dd_min_bid['Bid']
    min_bid_tid = arr_max_dd_min_bid['TID']

else:
    print("pass")

print(min_bid)
print(min_bid_tid)

'''

(0,)
(140,)
0
[(112.265, 112.290, 118772) (112.265, 112.288, 118773)
 (112.265, 112.287, 118774)]
1 0
1 3


'\nif np.squeeze(np.take(max_dd_min_bid, np.where(max_dd_min_bid[\'Bid\']>= max_dd_criteria))).size == 0:\n    print(np.argmax(max_dd_min_bid[\'Bid\']))\n    arr_max_dd_min_bid = max_dd_min_bid[np.argmax(max_dd_min_bid[\'Bid\'])]\n    min_bid = arr_max_dd_min_bid[\'Bid\']\n    min_bid_tid = arr_max_dd_min_bid[\'TID\']\n\nelse:\n    print("pass")\n\nprint(min_bid)\nprint(min_bid_tid)\n\n'